<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/CharRNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Reference Source : wikidocs.net 유영준님 자료

스스로 학습하면서 필요한 부분에는 추가적 설명, 소스 코드 삽입 및 수정 등이 있습니다. 영리적 목적이 아닌, 자기 계발 목적으로 정리한 자료입니다.

In [25]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [26]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

In [27]:
text

"\nI get on with life as a programmer,\nI like to contemplate beer.\nBut when I start to daydream,\nMy mind turns straight to wine.\n\nDo I love wine more than beer?\n\nI like to use words about beer.\nBut when I stop my talking,\nMy mind turns straight to wine.\n\nI hate bugs and errors.\nBut I just think back to wine,\nAnd I'm happy once again.\n\nI like to hang out with programming and deep learning.\nBut when left alone,\nMy mind turns straight to wine.\n"

In [28]:
#전처리
tokens = text.split() # '\n' 제거
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [29]:
# 글자 집합 만들기
char_vocab = sorted(list(set(text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [30]:
vocab_size=len(char_vocab)
print('글자 집합의 크기: {}'.format(vocab_size))

글자 집합의 크기: 33


In [31]:
#글자에 정수 인덱스 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [56]:
length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [58]:
sequences[:50]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a',
 'ith life as',
 'th life as ',
 'h life as a',
 ' life as a ',
 'life as a p',
 'ife as a pr',
 'fe as a pro',
 'e as a prog',
 ' as a progr',
 'as a progra',
 's a program',
 ' a programm',
 'a programme',
 ' programmer',
 'programmer,',
 'rogrammer, ',
 'ogrammer, I',
 'grammer, I ',
 'rammer, I l',
 'ammer, I li',
 'mmer, I lik',
 'mer, I like',
 'er, I like ',
 'r, I like t',
 ', I like to',
 ' I like to ',
 'I like to c',
 ' like to co',
 'like to con',
 'ike to cont',
 'ke to conte',
 'e to contem',
 ' to contemp',
 'to contempl',
 'o contempla',
 ' contemplat',
 'contemplate',
 'ontemplate ',
 'ntemplate b',
 'template be']

In [34]:
sequences[30:45]

['mmer, I lik',
 'mer, I like',
 'er, I like ',
 'r, I like t',
 ', I like to',
 ' I like to ',
 'I like to c',
 ' like to co',
 'like to con',
 'ike to cont',
 'ke to conte',
 'e to contem',
 ' to contemp',
 'to contempl',
 'o contempla']

In [35]:
#char_to_index를 사용하여 전테 데이터에 대해 정수 인코딩 수행
X = []
for line in sequences:
  temp_X = [char_to_index[char] for char in line]
  X.append(temp_X)

In [36]:
for line in X[:5]:
  print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [37]:
#예측 대상 글자 분리하기 - 모든 샘플 문장의 마지막 글자 분리시키기
sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1]

for line in X[:5]:
  print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [38]:
print(y[:5])

[18 28 17  0 21]


In [39]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) # y에 대한 원-핫 인코딩

In [43]:
print(X.shape) #샘플 개숫, 문장 길이, 글자 집합 갯수
print(X.shape[1])

(426, 10, 33)
10


In [42]:
X[5][:][:]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [44]:
#모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=0)

In [51]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
  init_text = seed_text
  sentence = ''

  for _ in range(n):
    encoded = [char_to_index[char] for char in seed_text] #현재 시퀀스에 대한 정수 인코딩
    encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
    encoded = to_categorical(encoded, num_classes=len(char_to_index))
    result = model.predict_classes(encoded, verbose=0) #입력한 X 시퀀스에서 y를 예측하고 y를 result에 저장

    for char, index in char_to_index.items():
      if index == result: # 예측한 글자와 동일하다면
        break
    seed_text=seed_text + char #현재 시퀀스 + 예측 글자를 현재 시퀀스에 추가
    sentence = sentence + char #예측 글자를 문장에 저장

  sentence = init_text + sentence
  return sentence

In [52]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to contemplate beer. But when I stop my talking
